# Análisis Exploratorio de Datos (EDA) - Amazon Sales Report

Este notebook tiene como objetivo realizar un Análisis Exploratorio de Datos (EDA) sobre el dataset **Amazon Sales Report**.  
Las metas principales son:
- Limpiar y preprocesar el dataset para garantizar consistencia.
- Obtener estadísticas descriptivas y detectar valores atípicos o inconsistentes.
- Generar nuevas variables que permitan un mejor análisis.
- Identificar patrones e insights relevantes para la operación de la tienda online.
- Exportar datasets preparados para herramientas de visualización como Tableau.

El análisis visual detallado se complementará en **Tableau**, mientras que en este documento se concentra la limpieza, auditoría, análisis estadístico y hallazgos clave.

In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

RAW_PATH = 'Amazon Sale Report.csv'

In [2]:
df = pd.read_csv(RAW_PATH, low_memory=False)
print('Shape RAW:', df.shape)
df.head()

Shape RAW: (128975, 24)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,"400,081.00",IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,"560,085.00",IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,"410,210.00",IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,"605,008.00",IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,"600,073.00",IN,NaN,False,NaN,NaN


## Limpieza y preprocesamiento

En esta sección se aplican transformaciones para asegurar la calidad de los datos:
- **Estandarización de nombres de columnas** para evitar errores por espacios o caracteres especiales.
- **Conversión de tipos de datos**: 
  - `Date` → tipo fecha.
  - `Qty`, `ship-postal-code` → enteros anulables.
  - `Amount` → numérico decimal.
- **Estandarización de campos categóricos**: se eliminan espacios, se convierten a *title case* y se corrigen inconsistencias en `ship-country`, `ship-state` y `ship-city`.
- **Manejo de duplicados**: se eliminan registros repetidos.
- **Detección de nulos**: se añade un flag (`Amount_is_null`) para facilitar el filtrado posterior.

Estas operaciones permiten disponer de un dataset homogéneo, robusto y listo para análisis posteriores.

In [3]:
# Copia de trabajo
df_clean = df.copy()

# Nombres de columnas
df_clean.columns = (
    df_clean.columns.str.strip()
                    .str.replace('\s+', ' ', regex=True)
)

# Drop columnas irrelevantes
cols_drop = [c for c in ['Unnamed: 22', 'promotion-ids'] if c in df_clean.columns]
if cols_drop:
    df_clean = df_clean.drop(columns=cols_drop)

# Tipos de datos
# Fecha
if 'Date' in df_clean.columns:
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%m-%d-%y', errors='coerce')

# Numéricos
for col in ['Qty', 'Amount', 'ship-postal-code', 'index']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        if col in ['Qty', 'ship-postal-code']:
            df_clean[col] = df_clean[col].astype('Int64')  # enteros anulables

# Strings comunes
for col in ['Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level', 'Style', 'SKU',
            'Category', 'Size', 'ASIN', 'Courier Status', 'currency', 'ship-city',
            'ship-state', 'ship-country', 'fulfilled-by', 'Fulfilled-By', 'Order ID']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()
        df_clean.loc[df_clean[col].isin(['nan','None','NULL','NaN','', ' ']), col] = np.nan

# Normalización de ubicaciones
def clean_location(s):
    if pd.isna(s): return s
    s = str(s).strip(" ,.;:-_/\\'\"()[]{}")  # quitar signos al borde
    s = re.sub(r'\s+', ' ', s)                   # colapsar espacios
    s = re.sub(r',\s*', ', ', s)                 # normalizar comas
    s = s.lstrip(', ').strip()
    if re.fullmatch(r'\d+(\.\d+)?', s) or len(s) <= 1:
        return np.nan
    return s.title()

for col in ['ship-city', 'ship-state', 'ship-country']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(clean_location)

# Duplicados
df_clean = df_clean.drop_duplicates()

print('Shape CLEAN:', df_clean.shape)
df_clean.head()


Shape CLEAN: (128975, 22)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B,fulfilled-by
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,Mumbai,Maharashtra,400081,In,False,Easy Ship
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,Bengaluru,Karnataka,560085,In,False,Easy Ship
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,Navi Mumbai,Maharashtra,410210,In,True,NaN
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,Puducherry,Puducherry,605008,In,False,Easy Ship
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,Chennai,Tamil Nadu,600073,In,False,NaN


## Auditoría de calidad de datos

El objetivo de esta sección es evaluar la confiabilidad de la información.  
Se realizan los siguientes pasos:
- Conteo de **valores nulos** por columna.
- Identificación de **fechas inválidas**.
- Reglas de consistencia:
  - `Qty <= 0` (cantidades no válidas).
  - `Amount < 0` (montos negativos).
  - Casos con `Qty = 0` y `Amount > 0`, o viceversa.
- Revisión de **diversidad de valores** en columnas críticas (`currency`, `Status`, `Courier Status`).

Los resultados permiten documentar las principales fuentes de error o ruido en los datos y tomar decisiones sobre su tratamiento.

In [4]:
quality_report = {}

# Nulos por columna (top 20)
quality_report['nulls_top20'] = df_clean.isna().sum().sort_values(ascending=False).head(20)

# Fechas inválidas
if 'Date' in df_clean.columns:
    quality_report['date_invalid'] = int(df_clean['Date'].isna().sum())

# Chequeos de consistencia
checks = {}

if 'Qty' in df_clean.columns:
    checks['qty_le_0'] = df_clean[df_clean['Qty'].notna() & (df_clean['Qty'] <= 0)].shape[0]

if 'Amount' in df_clean.columns:
    checks['amount_lt_0'] = df_clean[df_clean['Amount'].notna() & (df_clean['Amount'] < 0)].shape[0]

if 'currency' in df_clean.columns:
    checks['currency_unique'] = df_clean['currency'].dropna().nunique()

if 'Status' in df_clean.columns:
    checks['status_unique'] = df_clean['Status'].dropna().value_counts().head(10)

if 'Courier Status' in df_clean.columns:
    checks['courier_status_unique'] = df_clean['Courier Status'].dropna().value_counts().head(10)

quality_report['consistency_checks'] = checks
quality_report


{'nulls_top20': fulfilled-by        89698
 currency             7795
 Amount               7795
 Courier Status       6872
 ship-city              35
 ship-country           33
 ship-postal-code       33
 ship-state             33
 Sales Channel           0
 Fulfilment              0
 Order ID                0
 index                   0
 Status                  0
 Date                    0
 Qty                     0
 ASIN                    0
 Size                    0
 Category                0
 Style                   0
 SKU                     0
 dtype: int64,
 'date_invalid': 0,
 'consistency_checks': {'qty_le_0': 12807,
  'amount_lt_0': 0,
  'currency_unique': 1,
  'status_unique': Status
  Shipped                          77804
  Shipped - Delivered to Buyer     28769
  Cancelled                        18332
  Shipped - Returned to Seller      1953
  Shipped - Picked Up                973
  Pending                            658
  Pending - Waiting for Pick Up      281
  Shipped 

## Desarrollo de nuevas variables

Para enriquecer el análisis, se generan nuevas variables:
- **Variables temporales**: `Year_num`, `Month_str`, `Day_num`, `Dow`.
- **Indicadores de negocio**:
  - `UnitPrice_est = Amount / Qty` → precio unitario estimado.
  - `Order_Status_Bucket` → agrupa estados en “Successful”, “Unsuccessful” y “Other”.
  - `Sales_Channel_Bucket` → agrupa canales en “Online” y “Other”.
- **Flags de auditoría**: `Amount_is_null`, `qty_zero_with_amount`, `amount_zero_with_qty`.

Estas nuevas variables permiten explorar tendencias temporales, evaluar calidad de datos y facilitar la interpretación de resultados.


In [5]:
# Partes de fecha
if 'Date' in df_clean.columns:
    df_clean['Year_num']  = df_clean['Date'].dt.year
    df_clean['Month_str'] = df_clean['Date'].dt.to_period('M').astype(str)  # 'YYYY-MM'
    df_clean['Day_num']   = df_clean['Date'].dt.day
    df_clean['Dow']       = df_clean['Date'].dt.day_name()

# Precio unitario estimado
if {'Amount','Qty'}.issubset(df_clean.columns):
    df_clean['UnitPrice_est'] = np.where(
        (df_clean['Amount'].notna()) & (df_clean['Qty'].notna()) & (df_clean['Qty'] > 0),
        df_clean['Amount'] / df_clean['Qty'],
        np.nan
    )

# Bucket de estado
if 'Status' in df_clean.columns:
    status_ok = {'Delivered', 'Shipped', 'Shipped - Delivered to Buyer', 'Shipped - In Transit', 'Shipped - Out For Delivery'}
    status_bad = {'Cancelled', 'Refunded', 'Returned'}
    df_clean['Order_Status_Bucket'] = np.select(
        [
            df_clean['Status'].isin(status_ok),
            df_clean['Status'].isin(status_bad)
        ],
        ['Successful', 'Unsuccessful'],
        default='Other'
    )

# Canal simplificado
if 'Sales Channel ' in df_clean.columns:
    df_clean['Sales_Channel_Bucket'] = df_clean['Sales Channel '].str.strip().str.title()

# Flag de nulos en Amount
if 'Amount' in df_clean.columns:
    df_clean['Amount_is_null'] = df_clean['Amount'].isna()

df_clean.head()


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B,fulfilled-by,Year_num,Month_str,Day_num,Dow,UnitPrice_est,Order_Status_Bucket,Amount_is_null
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,Mumbai,Maharashtra,400081,In,False,Easy Ship,2022,2022-04,30,Saturday,NaN,Unsuccessful,False
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,Bengaluru,Karnataka,560085,In,False,Easy Ship,2022,2022-04,30,Saturday,406.00,Successful,False
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,Navi Mumbai,Maharashtra,410210,In,True,NaN,2022,2022-04,30,Saturday,329.00,Successful,False
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,Puducherry,Puducherry,605008,In,False,Easy Ship,2022,2022-04,30,Saturday,NaN,Unsuccessful,False
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,Chennai,Tamil Nadu,600073,In,False,NaN,2022,2022-04,30,Saturday,574.00,Successful,False


## Detección de outliers

Se aplicó el método **IQR (Interquartile Range)** para identificar valores atípicos en:
- `Amount`
- `UnitPrice_est`

Además, se agregan indicadores binarios (`*_is_outlier`) para marcar registros sospechosos.  
La detección temprana de outliers ayuda a distinguir entre:
- **Errores de carga** (ej: precios absurdamente altos/bajos).
- **Casos de negocio especiales** (ej: compras masivas poco frecuentes).

In [6]:
outlier_report = {}

def iqr_flags(s):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    return low, high

for col in ['Amount', 'UnitPrice_est']:
    if col in df_clean.columns:
        s = df_clean[col].dropna()
        if len(s) >= 10:
            low, high = iqr_flags(s)
            mask_low = df_clean[col] < low
            mask_high = df_clean[col] > high
            df_clean[f'{col}_is_outlier'] = mask_low | mask_high
            outlier_report[col] = {
                'low_threshold': float(low),
                'high_threshold': float(high),
                'n_outliers': int((mask_low | mask_high).sum())
            }

rules = {}
if {'Qty','Amount'}.issubset(df_clean.columns):
    rules['qty_zero_with_amount'] = int(((df_clean['Qty'].isna() | (df_clean['Qty']==0)) & df_clean['Amount'].gt(0)).sum())
    rules['amount_zero_with_qty'] = int(((df_clean['Amount'].isna() | (df_clean['Amount']==0)) & df_clean['Qty'].gt(0)).sum())

outlier_report['rules'] = rules
outlier_report


{'Amount': {'low_threshold': -59.5,
  'high_threshold': 1296.5,
  'n_outliers': 3600},
 'UnitPrice_est': {'low_threshold': -59.5,
  'high_threshold': 1296.5,
  'n_outliers': 3408},
 'rules': {'qty_zero_with_amount': 5136, 'amount_zero_with_qty': 2467}}

## Estadísticas descriptivas y frecuencias

Se calculan estadísticas básicas (media, mediana, desviación estándar, mínimo y máximo) para variables numéricas.  
Además, se muestran las categorías más frecuentes en variables clave (`Status`, `Fulfilment`, `ship-country`, etc.).

Este análisis permite:
- Detectar **tendencias centrales** y **variabilidad**.
- Identificar **categorías dominantes** que concentran la mayoría de las operaciones.

In [7]:
stats = {}

for col in ['Amount','Qty','UnitPrice_est']:
    if col in df_clean.columns:
        stats[col] = df_clean[col].describe()

display(pd.DataFrame(stats))

for col in ['Order_Status_Bucket','Status','Category','Fulfilment','Sales_Channel_Bucket','ship-city','ship-state','ship-country']:
    if col in df_clean.columns:
        print(f"\nTop valores en {col}:")
        display(df_clean[col].value_counts(dropna=False).head(15))

# KPIs
metrics = {}
if 'Amount' in df_clean.columns:
    metrics['Ingresos totales'] = float(np.nansum(df_clean['Amount']))
if {'Amount','Qty'}.issubset(df_clean.columns):
    metrics['Ticket promedio por línea'] = float(np.nanmean(df_clean['Amount']))
    metrics['Precio unitario promedio est.'] = float(np.nanmean(df_clean['UnitPrice_est']))
if 'Order_Status_Bucket' in df_clean.columns:
    total = len(df_clean)
    succ = int((df_clean['Order_Status_Bucket']=='Successful').sum())
    metrics['% líneas exitosas'] = round(100*succ/total, 2) if total else np.nan
metrics


,Amount,Qty,UnitPrice_est
count,"121,180.00","128,975.00","116,044.00"
mean,648.56,0.90,647.52
std,281.21,0.31,278.52
min,0.00,0.00,0.00
25%,449.00,1.00,449.00
50%,605.00,1.00,600.50
75%,788.00,1.00,788.00
max,"5,584.00",15.00,"2,598.00"



Top valores en Order_Status_Bucket:


Order_Status_Bucket
Successful      106573
Unsuccessful     18332
Other             4070
Name: count, dtype: int64


Top valores en Status:


Status
Shipped                          77804
Shipped - Delivered to Buyer     28769
Cancelled                        18332
Shipped - Returned to Seller      1953
Shipped - Picked Up                973
Pending                            658
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipping                             8
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

...


Top valores en ship-state:


ship-state
Maharashtra       22260
Karnataka         17326
Tamil Nadu        11483
Telangana         11330
Uttar Pradesh     10638
Delhi              6967
Kerala             6585
West Bengal        5963
Andhra Pradesh     5430
Gujarat            4489
Haryana            4415
Rajasthan          2711
Madhya Pradesh     2529
Odisha             2136
Bihar              2114
Name: count, dtype: int64


Top valores en ship-country:


ship-country
In     128942
NaN        33
Name: count, dtype: int64

{'Ingresos totales': 78592678.29999998,
 'Ticket promedio por línea': 648.5614647631621,
 'Precio unitario promedio est.': 647.5175536865327,
 '% líneas exitosas': 82.63}

## Insights preliminares

A partir del análisis exploratorio se identifican hallazgos clave:
- **Ciudades y categorías top** en términos de ingresos.
- **Meses con mayor actividad** y evolución temporal de ventas.
- **Porcentaje de pedidos cancelados** u operaciones no exitosas.
- **Número de registros outliers** en precios o cantidades.

Estos insights iniciales servirán como base para un análisis visual más detallado en Tableau.

In [8]:
insights = []

if {'Category','Amount'}.issubset(df_clean.columns):
    by_cat = df_clean.groupby('Category', dropna=True)['Amount'].sum().sort_values(ascending=False)
    if not by_cat.empty:
        insights.append(f"Top categoría por ingresos: **{by_cat.index[0]}** ({by_cat.iloc[0]:,.2f}).")

if {'ship-city','Amount'}.issubset(df_clean.columns):
    by_city = df_clean.groupby('ship-city', dropna=True)['Amount'].sum().sort_values(ascending=False)
    if not by_city.empty:
        insights.append(f"Top ciudad por ingresos: **{by_city.index[0]}** ({by_city.iloc[0]:,.2f}).")

if {'Month_str','Amount'}.issubset(df_clean.columns):
    monthly = df_clean.groupby('Month_str')['Amount'].sum().sort_index()
    if len(monthly) >= 2:
        base = monthly.iloc[0] if monthly.iloc[0] != 0 else np.nan
        if pd.notna(base):
            growth = (monthly.iloc[-1] - monthly.iloc[0]) / base
            insights.append(f"Crecimiento entre primer y último mes observado: **{growth*100:.1f}%**.")
        insights.append(f"Mes pico de ingresos: **{monthly.idxmax()}** ({monthly.max():,.2f}).")

if 'Status' in df_clean.columns:
    cancelled = (df_clean['Status']=='Cancelled').mean()*100
    insights.append(f"Porcentaje de líneas con **Cancelled**: **{cancelled:.2f}%**.")

for col in ['Amount','UnitPrice_est']:
    flag = f'{col}_is_outlier'
    if flag in df_clean.columns and df_clean[flag].any():
        insights.append(f"Se detectaron **outliers** en **{col}**: {int(df_clean[flag].sum())} filas (IQR).")

print("\n".join(insights) if insights else "No se generaron insights automáticos.")


Top categoría por ingresos: **Set** (39,204,124.03).
Top ciudad por ingresos: **Bengaluru** (7,259,693.80).
Crecimiento entre primer y último mes observado: **22937.9%**.
Mes pico de ingresos: **2022-04** (28,838,708.32).
Porcentaje de líneas con **Cancelled**: **14.21%**.
Se detectaron **outliers** en **Amount**: 3600 filas (IQR).
Se detectaron **outliers** en **UnitPrice_est**: 3408 filas (IQR).


## Exportación de datasets

Se generan dos archivos de salida:
- `Amazon_Sale_Report_CLEAN.csv` → con formato regional (separador `;`).
- `Amazon_Sale_Report_CLEAN_TABLEAU.tsv` → delimitado por tabulador, ideal para Tableau.

Esto asegura compatibilidad tanto para análisis reproducibles en Python como para visualizaciones dinámicas en Tableau.

In [9]:
# CSV regional-friendly (ES) y TSV (universal para Tableau)
df_clean.to_csv('Amazon_Sale_Report_CLEAN.csv', index=False, sep=';', decimal=',')
df_clean.to_csv('Amazon_Sale_Report_CLEAN_TABLEAU.tsv', index=False, sep='\t')

'Export listo.'

'Export listo.'

# Recomendaciones y decisiones tomadas

- **Normalización de datos**: se estandarizaron los nombres de ciudades, estados y países, eliminando inconsistencias de mayúsculas/minúsculas y valores no válidos.  
- **Manejo de valores nulos**: se decidió conservar los registros, pero se añadió un flag (`Amount_is_null`) para filtrar en análisis posteriores.  
- **Detección de outliers**: se marcaron casos atípicos sin eliminarlos, ya que pueden ser relevantes para el negocio.  
- **Nuevas variables**: se generaron columnas de fechas, precio unitario estimado y buckets de estado/canal para enriquecer el análisis.  
- **Consistencia de negocio**: se marcaron reglas lógicas (ej: cantidad cero con monto positivo) que deberán revisarse en la fuente de datos.  
- **Visualización en Tableau**: se exportaron datasets listos para análisis visual complementario.  

En conjunto, estas decisiones garantizan un **dataset robusto, confiable y analíticamente enriquecido**, que servirá como base sólida para análisis descriptivos y visualizaciones ejecutivas.